In [1]:
# 1609492611000, 1641028611000, 1672564611000 (2021, 2022, 2023) 1677632461000

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import pandas_ta as p_ta
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [21]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover, resample_apply

In [5]:
interval_arr = ['15m', '1h', '4h', '1d']
start_time_arr = [1677632461000]

In [6]:
timezone = 8
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

start_time = 1679274061000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds
step = 60000 * 3600

In [7]:
def create_raw(symbol, interval_arr, start_time, end_time, step):
    
    url = "https://api.binance.com/api/v3/klines"
    
    for interval in interval_arr:

        raw_df = pd.DataFrame()
        
        for timestamp in range(start_time, end_time, step):
            params = {"symbol": symbol_C,
                      "interval": interval,
                      "startTime": timestamp,
                      "endTime": timestamp + step}
            response = requests.get(url, params=params).json()
            out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                                   "Volume", "Close_Time", "Quote asset volume",
                                                   "Number of trades", "Taker buy base asset volume",
                                                   "Taker buy quote asset volume", "Ignore"])
            raw_df = pd.concat([raw_df, out], axis = 0)

        raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]

        raw_df.to_hdf(f'klines_{symbol}_{interval}.h5', key='df', mode='w')
        print(f"Created {symbol}_{interval}")

In [8]:
loop_start_time = time.time()
create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

Created ethusdt_15m
Created ethusdt_1h
Created ethusdt_4h
Created ethusdt_1d
Time taken to execute for loop: 8.050949573516846 seconds


In [9]:
h5 = pd.read_hdf(f'klines_{symbol}_{interval_arr[-1]}.h5', key='df')
print(h5.tail(1))

      Close_Time           Open          Close           High            Low  \
0  1681257599999  1910.21000000  1918.66000000  1937.37000000  1906.25000000   

            Volume  
0  232101.56650000  


In [10]:
def time_format(timezone):
#     df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms') + pd.Timedelta(hours=timezone)
#     df['Close_Time'] = df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
#     df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')

In [72]:
klines_cache = {}

def get_klines(symbol, interval, start_time, end_time):
    if (symbol, interval) not in klines_cache:
        klines_cache[(symbol, interval)] = pd.read_hdf(f'klines_{symbol}_{interval}.h5', key='df')

    df = klines_cache[(symbol, interval)].query(f"Close_Time >= {start_time} and Close_Time <= {end_time}")

    df = df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)

    df = df.set_index('Close_Time')
#     df = df.reset_index(drop=True)

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)

    print(df)

    return df

In [73]:
cash = 20000
leverage = 10
commission = 0.05/100

In [78]:
class RsiOscillator(Strategy):
    
    rsi_up = 70
    rsi_low = 30
    rsi_window = 14
    
    kd_int = 14
    d_int = 3
    
    ema1 = 8
    ema2 = 18
    ema3 = 38

    # Do as much initial computation as possible
    def init(self):
        
        self.data = self.data.set_index('Close_Time')
        close_series = pd.Series(self.data.Close)
            
        self.rsi_15m = self.I(p_ta.rsi, close_series, self.rsi_window)
        self.rsi_1h = resample_apply('H', p_ta.rsi, close_series, self.rsi_window)
        
        self.ema1 = self.I(ta.trend.ema_indicator, close_series, window=self.ema1)  
        self.ema2 = self.I(ta.trend.ema_indicator, close_series, window=self.ema2)   
        self.ema3 = self.I(ta.trend.ema_indicator, close_series, window=self.ema3)   

        
        kd = ta.momentum.StochasticOscillator(high = pd.Series(self.data.High),
                                              low = pd.Series(self.data.Low),
                                              close = close_series,
                                              window = self.kd_int,
                                              smooth_window = self.d_int)
        
        self.k = self.I(kd.stoch)
        self.d = self.I(kd.stoch_signal)

    # Step through bars one by one

    def next(self):
        if (self.rsi_15m[-5:] > self.rsi_up).any() and crossover(self.d, self.k):
            self.position.close()
        elif (self.rsi_15m[-5:] < self.rsi_low).any() and crossover(self.k, self.d):
            self.buy(size = 0.1)

bt = Backtest(get_klines(symbol, interval, start_time, end_time),
             RsiOscillator,
             cash = cash,
             commission = commission,
             margin = 1/leverage)

stats = bt.run()
print(stats)
bt.plot(filename = 'backtesting.html')

                                  Open    Close     High      Low     Volume
2023-03-20 09:14:59.999000064  1771.42  1777.05  1778.59  1761.62  9531.7973
2023-03-20 09:29:59.999000064  1777.06  1777.17  1778.70  1773.93  3494.7823
2023-03-20 09:44:59.999000064  1777.17  1777.98  1779.40  1774.04  4846.9503
2023-03-20 09:59:59.999000064  1777.98  1774.79  1779.20  1771.16  4508.2285
2023-03-20 10:14:59.999000064  1774.78  1779.04  1779.82  1771.58  6104.2944
...                                ...      ...      ...      ...        ...
2023-04-11 21:29:59.999000064  1913.72  1916.61  1916.62  1913.10  2274.4557
2023-04-11 21:44:59.999000064  1916.62  1913.55  1917.81  1912.02  3805.2792
2023-04-11 21:59:59.999000064  1913.55  1920.47  1921.88  1913.06  6662.6918
2023-04-11 22:14:59.999000064  1920.46  1918.30  1922.91  1917.78  4724.8478
2023-04-11 22:29:59.999000064  1918.29  1919.90  1920.74  1915.86  3235.7168

[2161 rows x 5 columns]


AttributeError: Column 'set_index' not in data

In [61]:
from backtesting.test import GOOG
print(pd.Series(GOOG.Close))

2004-08-19    100.34
2004-08-20    108.31
2004-08-23    109.40
2004-08-24    104.87
2004-08-25    106.00
               ...  
2013-02-25    790.77
2013-02-26    790.13
2013-02-27    799.78
2013-02-28    801.20
2013-03-01    806.19
Name: Close, Length: 2148, dtype: float64
